## Installing required libraries

In [1]:
%%capture
!pip install gradio transformers torch soundfile librosa

In [2]:
import librosa
import torch

def chunk_audio(audio, chunk_duration, overlap_duration, sample_rate=16000):
    # Convert duration from seconds to samples
    chunk_samples = int(chunk_duration * sample_rate)
    overlap_samples = int(overlap_duration * sample_rate)

    chunks = []
    # Iterate through the audio in steps, subtracting the overlap to ensure continuity
    for start in range(0, len(audio) - overlap_samples, chunk_samples - overlap_samples):
        end = start + chunk_samples
        chunk = audio[start:end]
        chunks.append(chunk)
    return chunks

## Interface for main model

### For short segments upto 3-4 seconds

In [ ]:
%%capture
!git lfs install

In [ ]:
# !git clone https://huggingface.co/Harveenchadha/vakyansh-wav2vec2-nepali-nem-130

In [ ]:
import gradio as gr
from transformers import pipeline, AutoProcessor, AutoModelForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import torch
import soundfile as sf
import librosa

# Initializing the ASR pipeline using our model
# pipe = pipeline("automatic-speech-recognition", model="iamTangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
# model_name='iamTangsang/XLS-R-300m-Nepali-CommonVoice'
model_name='iamTangsang/Wav2Vec2_XLS-R-300m_Nepali_ASR'
# model_name='iamTangsang/Final_XLS-R-300m_Nepali'

processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)
# Processor and model in case you want to work directly with them
# processor = AutoProcessor.from_pretrained("iamTangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
# model = AutoModelForCTC.from_pretrained("iamTangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
# processor = AutoProcessor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-nepali-nem-130")
# model = AutoModelForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-nepali-nem-130")
# Use a pipeline as a high-level helper

# pipe = pipeline("automatic-speech-recognition", model="Harveenchadha/vakyansh-wav2vec2-nepali-nem-130")
# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token=UNK_TOKEN, pad_token=PAD_TOKEN, word_delimiter_token=WORD_DELIMITER)
# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/vakyansh-wav2vec2-nepali-nem-130")
# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000,
#                                              padding_value=0.0, do_normalize=True,
#                                              return_attention_mask=True)


# Function to process speech and return the transcription
def transcribe_speech(audio):
    # Read the audio file
    speech, sr = sf.read(audio)
    if sr != 16000:
      speech = librosa.resample(speech, orig_sr=sr, target_sr=16000)

    # Ensure the audio is in the correct format
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

    # Generate logits and decode using the processor
    with torch.no_grad():
        logits = model(**inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)

    # transcription = pipe(speech)

    return transcription[0]
    # return transcription['text']

# Gradio interface
iface = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(type="filepath"),  # Use the default for uploading files or recording from the microphone
    outputs="text",
    title="Nepali Speech Recognition",
    description="Upload or record a Nepali speech to transcribe."
)

# Launch the Gradio app
iface.launch(debug=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c7fc9b0d9502a53264.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

## Chunking long audio files to segments for longer audios

### Gemini as language model. Using only temporarily

In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 19.3 MB/s eta 0:00:00


### Installing genai

In [ ]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model_gemini = genai.GenerativeModel("gemini-1.5-flash")
response = model_gemini.generate_content("Write a story about a magic backpack.")
print(response.text)

### Main Interface

In [ ]:
import librosa
import torch

def chunk_audio(audio, chunk_duration, overlap_duration, sample_rate=16000):
    # Convert duration from seconds to samples
    chunk_samples = int(chunk_duration * sample_rate)
    overlap_samples = int(overlap_duration * sample_rate)

    chunks = []
    # Iterate through the audio in steps, subtracting the overlap to ensure continuity
    for start in range(0, len(audio) - overlap_samples, chunk_samples - overlap_samples):
        end = start + chunk_samples
        chunk = audio[start:end]
        chunks.append(chunk)
    return chunks


In [ ]:
import gradio as gr
from transformers import pipeline, AutoProcessor, AutoModelForCTC
import torch
import soundfile as sf
import librosa

# Initialize the ASR pipeline using your model
# gagan3012/wav2vec2-xlsr-nepali
# iamTangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr
# Strange18/wav2vec2-nepali-asr
# spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr
# pipe = pipeline("automatic-speech-recognition", model="iamTangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

# Processor and model in case you want to work directly with them
# processor = AutoProcessor.from_pretrained("iamTangsang/XLS-R-300m-Nepali-CommonVoice")
# model = AutoModelForCTC.from_pretrained("iamTangsang/XLS-R-300m-Nepali-CommonVoice")
model_name = 'iamTangsang/Wav2Vec2_XLS-R-300m_Nepali_ASR'
# model_name = 'Harveenchadha/vakyansh-wav2vec2-nepali-nem-130'
# model_name='iamTangsang/Final_XLS-R-300m_Nepali'
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCTC.from_pretrained(model_name)

# Function to process speech and return the transcription
def transcribe_speech(audio):
    # Read the audio file
    speech, sr = sf.read(audio)
    if sr != 16000:
      speech = librosa.resample(speech, orig_sr=sr, target_sr=16000)
    duration = librosa.get_duration(y=speech, sr=16000)

    if duration > 10:
      chunk_duration = 5  # Duration of each chunk in seconds
      overlap_duration = 0.05  # Overlap duration in seconds

      # Load your 30-second audio
      # Assuming input_audio is a NumPy array or a list of audio samples
      # input_audio, _ = librosa.load("path/to/30-second-audio.wav", sr=16000)

      # Create chunks of the audio
      audio_chunks = chunk_audio(speech, chunk_duration, overlap_duration)
      # Store predictions
      predictions= []
      # Perform inference on each chunk
      for chunk in audio_chunks:
      # Prepare the input for each model
        input_dict = processor(chunk, sampling_rate=16000, return_tensors="pt", padding=True)
        logits= model(input_dict.input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)[0]
        predictions.append(processor.decode(pred_ids, skip_special_tokens=True))

      # Combine predictions
      full_prediction= " ".join(predictions)
      # response = model_gemini.generate_content("Behave like 5 gram Nepali language model. Correct only the words. Remove duplicates. Don't use '।' and punctuations. the following transcription in Nepali language: " + full_prediction)
      # full_prediction = response.text
      # print("Our Model: " + full_prediction)
      # # response = model_gemini.generate_content("The following is the transcription by my speech recognition model. Correct the words i.e. replace the meaningless words with most appropriate ones but don't remove any words. Split words where necessary.  Add nepali full stop appropriately. Give output like speech recognition model. No line breaks: " + full_prediction)
      # full_prediction = response.text
      # print("Corrected by Gemini: " + full_prediction)

      return full_prediction

  # Ensure the audio is in the correct format
    inputs = processor(speech, sampling_rate=16000, return_tensors="pt", padding=True)

  # Generate logits and decode using the processor
    with torch.no_grad():
      logits = model(**inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    # print("Our Model: " + transcription[0])

    # response = model_gemini.generate_content("The following is the transcription by my speech recognition model. Correct the words i.e. replace the meaningless words with most appropriate ones but don't remove any words. Split words where necessary.  Add nepali full stop appropriately. Give output like speech recognition model. No line breaks: " + transcription[0])
    # # response = model_gemini.generate_content("Behave like 5 gram Nepali language model. Correct only the words. Remove duplicates. Don't paraphrase. Don't use '।' and punctuations. the following transcription in Nepali language: " + transcription[0])
    # # transcription[0] = response.text
    # transcription[0] = response.text
    # print("Corrected by Gemini: " + transcription[0])

    return transcription[0]

# Gradio interface
iface = gr.Interface(
    fn=transcribe_speech,
    inputs=gr.Audio(type="filepath"),  # Use the default for uploading files or recording from the microphone
    outputs="text",
    title="Nepali Speech Recognition",
    description="Upload or record a Nepali speech to transcribe."
)

# Launch the Gradio app
iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9c90dab6886bdf4187.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1520, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

### Testing the effectiveness of Gemini-1.5-flash

In [ ]:

# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
text = '''

नन कैनल का नेपाल सन्दर्भबाट सन जीदिरको नमस्कार आाम बिशवव्यापीरूपमा फैलिरहेको कुरुणाभइरतसङ्क्रमण ्रभाव नेपालमा पनिदेखिएको छ परीक्षणको दाय यसँगै नेपालमा पछिल्लाकेीहकप्तादेखि सङ्क्रमि सङ्ख्यामा व्यापक वृद्धि देखिको छ कोभिड नाइन्टी मूलतस्वास्थसँग सम्बन्धित समस्यार कारणयसको प्रभावसङ्करमित र त्यसबाट मृत्यु ारूपमा अघिआएको भएपनि आर्थिक समाजिक अन्य विभिन्न क्षेत्रमासमेत यसको दुर्गा रावरहनी बताइएको छ यसैसन्दर्भमा केहीदिनअघि हामीले नेपालका लागि राष्ट्रसङ्घी आवासीयस बालरी जुलियनसँग कोपिडनाइटिनमहा रिकाबीच नेपालको पछिल्लो अवस्था सम्भावितप्रभाव र नेपालमा केन्द्रित रहेर रससङघको पहललगायका वि कुराकानिकरेकाछौँ सुरुमा हामीले उनलाई कुरनाभइरस महामारिकावीचपछिल्लो विश्वव्यापी अभ र नेपालमा त्यसको प्रभावकस्तो देखिन्छ भनेर प्रश्न गरेका थियँ उपतपयाफे हामीले अहिले जीव भोगिरहेकाछौँ त्यो अभूतपूर्वसंकट होर त्यसक दुईटाबायो अभूतपूर्व भएकाले हामी ग यो महानारीसघ षङ्घर्षघर्न कुनैपनि किसिमक थापितविधि छैनयो एउटा स्वास्यसँग सम्बन जसले मानिसहरूको स्वास्थ्यमा समस् तर योभाहिरसङघ स्कर्षका लाग अवलम्बन गरिएका उपायहरूले स्वास्थ्यमात् नभएर विभिन्न क्षेत्रमा प्रभावतारिरहेको छ नेपालको विषयमा कुरा गर्नुपर्था यो आर्थिकवर्षका लागि बृद्धिदर परिमार्जन गरेर सातबाट करिब दुईवुदशमलब दुईप्रतिशत कायन गरिएको छ जसको अर्थ अर्थतन्त्रमा रहसाउनेछयो सङकटले पहिलेदेखिने काय असमानतालाई बढाइदिएको हामीले त्यो महिलाहरूमा देखिरहेकाछौँआफूमा दुर्व्यवहारहुँदासमेत उनीहरूले त्यस्तो गर्ने व्यक्ति ैबस्नबाध्यछन् नेपालजस्ता देशहरूले पछिल्लो दुईदशकदेखि सहश्राप्ती विकासलक्षाएम का उद्देश्य प्राप्त गर्न र हालदिगो विकास लक्षको उद्देश्य प्राप्त गर्न ठुलोप्रयास गरेका छ र हामीले नेपाल तथा अरुदेशमा प्राप्तग िएका उपलब्धिसमेत गुम्नसक्ने सम्भावन कुल्णाभारतको विश्वव्यावी महामारीकाव संयुक्तराष्ट्रसङ्घले नेपाललाई किकस्ता सहयोग गरिरहेको छ यो महामारी अन्तभएपछिको अवस्थाका लागि केक ानीतितहरजुमा गरिएको छ पयाट हामीले अहिले स सङ्कटस सङ्घर्ष गर्नका लागि आवश्यकत मा सादिरहेका खासगरीस्वास्थ्य क्षेत्रमा मलाई केलाग्छ भने अतिकमविकसित बुलुकहरू रहेको नेपालजस्तो मुलुक जहाँ अझैपनि स्रोत रसाधनको कमिछ त्यहाँ अहिले उनलाइसभन्दा बढी प्रयोगशाला बनिसकेका छन्सबैप्रदेशमा अस्पताल बनेकछन यो एकपरकारको प्रगतिहोर ामीले त्यसमा सहयोग गरिरहेकासँकोविटउड्नाइसको पछिको अवस्थाको कुरागर्दा हामीले कुन क्षेत्रमा बढीप्रभा पर्छभन्ने कुरा बुझ्नजहरूरीछ हामीले अहिलेको यो महामारी अघिनै असुरक्षित रहेकाहरूको पछिल्लो अवस्थाको मुल्याङकन गरिरहेकाछौँ अनपचा दैनिक ज्यालादार मजदूरीमा बढीप्रभावदेखि गरिभन्न सकिने थप मानिसहरूदे उदाहरणका लागि विदेशमा रोजगारी गरी देशफर्किनेछन् जब उनीहरू यहाँ आउँछ उनीहरूसँग रोजगारीको अवसर हुनेछैनउनी रिवारजसको आन्धामीको श्रोतविप्रेषण हुन्थ्यो उनीहरू पनि गरिबकै श्रेणीमा पर्नेछन्हामि ेरिरहेको अर्को क्षेत्रभनेको साना मझौला तथा द्यन लामोलगरङका कारणतीमत धेरै टाटपल्टिनलागेका हुनसक्छन् र आ तिकसंकटको सामना गर्न असक्षम हुनसक्छन् बारे प्रभावअध्ययन गरेर हामीले अहिलेसरकार योजना निर्माणमा काम गरिरहेकाछौँतर कुरानिश्चित छ भने हामीले पहिलेजस्तै गरी क्रिया भने जनाउन सक्दैन किनकि कोविट उन्डाइ ले हाम्रो प्रणालीका धेरै कमजोरबाटहरू देखाइदिएको यसको अर्थअफकादिनमा राष्ट्रसङ्ख्य निका ध्यानकेही क्षेत्रमा मात्रैबढीकेन्द्रत हुने भनयो बजेरदेखिलिएर धेरै समस्यादेखेकासन् नेपालमा संयुक्तरराष्ट्रसङ्घलेगर्दै आएका कामहरूमा केही परिवर्तन हुनेछ आइन्डट औनविश्रयाक्तिक्ष मेरो विचारमा हामी सबै बदलिनजरूेछ सन् माधिमहरू परिवर्तन हुनजरूिछ समृ्तराष्ट्र ा परिवर्तन हुनजरूरीछ र सरकारहरूपरिवर् नेपालजस्तो देशमा व्यक्तिहरूमा निकै महत्त्वपूर्ण आर्थिक क्षेत्रमा रत्यसमध्ये एउटा क्षेत्रपर्ि क्षेत्रहो जसमा अर्चालीसतसमलव एकाउन्न अर्भर यहाँ भन्दा बढीको रहसाएको छमा क्षेत्रमा उने पदयात्रामा मातृजोडिएकाहरूको सङ्ख्या पन्ध्रलागभन्दा बढी छ र उनीहरूले रोजगारी त्यसकारण हामी सबै परिवर्तन हुनजरु राष्ट्रसङ्घमा परिवर्तन नहुने कुरा भनेको िकोविकासको लक्ष हो किनभने यसमा हामीले अहि ेखिरहेका समस्याका समाधानसमेत छन् दिकोविका कसैलाई पनि नछोड्न समावेशी हुनु हुनु र वातावरणको सम्मान गर्ने समाजनिर्माणका आवश्यक प्रणाली विकास गर्नुलगायतका विशेषमा कोविड उन्नाइस महामारी लेकेकुरा देखाएक हामीलेगरेको विकासदिको छैन र न्यायस जसकारण हामी बदलिनजरुरी छ राष्ट्रसङघले त्यसतर्फ खबरदारी गरिनै रहनेछ अहिलेसम्म नेपालबारे संयुक्त राष्ट्रसङघलेकरेको अध्यन र ककृससाइटक्रिकरिवयन एतय संयुक्तराष्ट्रसङ्घको तर्फबाट दिभि ध्ययनभएकाछ त्यसका निचोडभारी मैले भरखरी संयुक्तराष्ट्रसङ्घीय विकासकार्यक्रम बेन्डिपीको कोविड उन्नाइसको सङ्क्रमणका कारणआ कसमाजिक क्षेत्रमा परेको प्रभावबारी प्रथममूल्याङ लेमात्र तयारपारेको छ त्यसमा पर्यटनर ृषिलगायतका विषय समेटिएका छन् उक्त प्रतिवेदनमा समस्यामा रहेका क्षेत्रहरू पुरानैलयमा फर्किनन पच्छीप्रतिशतभन्दा बढीरोजगारी कटौती हुन निजी क्षेत्रको अनुमान रहेको उल्लेख गरिएको छ अविकसितराष्ट्रमा सूचीकृत नेपालका आफ्नो किसिमकास र कोवीर उन्नाइस महामारीलेतीसमस्याजन नेपालमा सबैभन्दा बढी प्रभावकुलकु क्षेत्रदेखिन्छ र प्राथमिकतामा राख्नुपर्ने विशेके वतघेट क्रियय संयुक्त राष्ट्रसङ्घको हसतक्षेुने तिनवटा क्षेत त्यसमा एउटा सुवास्तय होर हामीले अहिले ्वास्यसँग सम्बन्धित सङकटबाट बाहिरिन सकेका छैनौँ त्यसकारण हामिस्वासी क्षेत्रमा निरन्तर काम गरिरहेका छ हाम्रो ध्यान सङ्क्रमणफैलिन नदिनेमा केन्द्रित हुन टाकी अस्पतालहरूमा चापकमहोस् नेपालको प्रणालले अधिकतम दशहजारसङ्क्रमितमा तरधनसक्ने अवस्था रहेको स्वीकार गरेको छदश रभन्दाबढीमानिसङ्क्रमितभए नेपालका लागि निकैघार दोस्रोबाटो भनेको स्वास्यबाहेका अन्य मानवीय विषय हुहामीले पोषण पानीर फाई र खादी सामक्रीवितरणजस्ता विषयमा ध्यानदिन तेस्रोबाटो भनेको आर्थिक समाजिक क्षेत्र र यसमा हामीले विभिन्न तहबाट नियाली रहेका पिछडिएका वर्गको क्षमता अभिविृद्धि गरेर रोजगारीको अवसर सृजना गर्नुपर्नेछ विदेशता फर्किएका श्रमीकहरूको शिपको उचितप्रयोग गर्ने विषय हत्त्वपूनेपालको असिप्रतिशतभन्दा बढी ज प्रत्यक् वा अप्रत्यक्ष रूपमा कृषि क्षेत जोडिएको अवस्थामा यो क्षेत्रको विकासपनि िामा पर्नुपर्छ त्यो किनपनि भने यो क्षेत् मा धेरै कामदारहरूको आवश्यकता हुन्छ रयसले ने पालको आत्मनिर्भरता पनि बढाउँछहामीकर अन्त्यतिर आइसकेकाछँ तपाईँले कोविर उन्नाइस प्रभावनियन्त्रणमा खटिएका उच्च अधिकारीहरूसँगै नेतृत्वक ग पनि नियमित भेटघाट गरिरहनुभएको छउनी गकुराकानीगर्दा सरकारको तयारी कस्तो भउ्नुभएको छ यवितकती कुरिङगुधविनिश्व साइन हामीले आर्थिक समाजिक बाटोभारे अर्थमन्त्रालयल गायतका निकायहरूसँग छलफल गरिरहेका सँसंी ाष्ट्रसङसहित अन्य अन्तर्राष्ट्रियवित्ीयसंस्थाहरूको त सवलिकरण र पुनरुथानसम्न योजना बनाउने काम पनि भइरहेको छ ा नेपाल अन्तर्राष्ट्रिय जगतबाट सल्लाहरसिफारिस लि ुकदेखिन्छतर कसैसँग पनि चमत्का उत्तछैनकिनभने प्रत्येक स्थिति फरक छ त्यसकारणमुक के कुरादोखराउन चाहन्छु भने पालमा संयुक्तराष्ट्रसँघले सामाजिक क्षेत्रलाईसर योजना र त्यसबाहेपनि प्राथमिकता पाओसभन् यमा लागिनै रहनेछ पुते् ुटै कमि इन्डवेसलेक निन्टो फ्लान अफ्टककेनेट नेपालका लागि संयुक्तराष्ट्रसङ्खीय आवासीय संयोज जुलियनहस्थआजका लागि नेपालसन् ब यति नै

'''
model_gemini = genai.GenerativeModel("gemini-1.5-flash")
response = model_gemini.generate_content("The following is the transcription by my speech recognition model. Correct the words i.e. replace the meaningless words with most appropriate ones but don't remove any words. Split words where necessary.  Add nepali full stop appropriately. Give output like speech recognition model. No line breaks: " + text)
print(response.text)

नन क्यानल का नेपाल सन्दर्भबाट सन जीदिरको नमस्कार ।  आाम विश्वव्यापीरूपमा फैलिरहेको कोरोनाभाइरस सङ्क्रमणको प्रभाव नेपालमा पनि देखिएको छ ।  परीक्षणको दायरा यसँगै नेपालमा पछिल्ला केही हप्तादेखि सङ्क्रमित सङ्ख्यामा व्यापक वृद्धि देखिएको छ ।  कोभिड १९ मूलतः स्वास्थ्यसँग सम्बन्धित समस्याको कारण यसको प्रभाव सङ्क्रमित र त्यसबाट मृत्यु हुनेहरूमा अघि आएको भएपनि आर्थिक, सामाजिक अन्य विभिन्न क्षेत्रमा समेत यसको दुर्गा रावरहनी बताइएको छ ।  यसै सन्दर्भमा केही दिन अघि हामीले नेपालका लागि राष्ट्रसङ्घीय आवासीय संयोजक जुलियनसँग कोभिड १९ महामारीबीच नेपालको पछिल्लो अवस्था, सम्भावित प्रभाव र नेपालमा केन्द्रित रहेर राष्ट्रसङ्घको पहल लगायतका विषयमा कुराकानी गरेका छौँ ।  सुरुमा हामीले उनलाई कोरोनाभाइरस महामारीबीच पछिल्लो विश्वव्यापी अवस्था र नेपालमा त्यसको प्रभाव कस्तो देखिन्छ भनेर प्रश्न गरेका थियौँ ।  उपत्यकामा हामीले अहिले जिवन भोगिरहेका छौँ, त्यो अभूतपूर्व सङ्कट हो ।  त्यसको दुईटा बायो अभूतपूर्व भएकाले हामी यो महामारीसँग षङ्घर्ष गर्न कुनैपनि किसिमको थापित विधि छैन ।  यो एउटा स्वास्थ्यसँग सम्बन्धित समस्या 

# Character Error Rate Calculation

In [ ]:
import Levenshtein

def compute_cer(reference, hypothesis):
    # Remove spaces or other non-alphanumeric characters if necessary
    reference = reference.replace(" ", "")
    hypothesis = hypothesis.replace(" ", "")

    # Compute the edit distance
    edit_distance = Levenshtein.distance(reference, hypothesis)

    # Compute CER
    cer = edit_distance / len(reference)
    return cer

reference_text = "hello world"
hypothesis_text = "hallo wrld"
cer = compute_cer(reference_text, hypothesis_text)
print(f"Character Error Rate (CER): {cer:.4f}")


ModuleNotFoundError: No module named 'Levenshtein'

# Rubbish

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="sherwinseah/Fine-tuned-T5-for-MCQGenerator")
